<div align="center" style="font-size: 2rem">

<img heoght="300px" src="https://cta-observatory.github.io/ctapipe/_images/ctapipe_logo.png" alt="ctapipe"/>


<p style="text-align: center;">LST Analysis Bootcamp</p>

<p style="text-align: center">Padova, 26.11.2018</p>

<p style="text-align: center">Maximilian Nöthe (@maxnoe) & Kai A. Brügge (@mackaiver)</p>

</div>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 14

plt.rcParams['figure.figsize']

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

# General Information

## Design

* DL0 → DL3 analysis

* Currently some R0 → DL0 code to be able to analyze simtel files

* ctapipe is built upon the Scientific Python Stack, core dependencies are
  * numpy
  * scipy
  * astropy

## Developement

* ctapipe is developed as Open Source Software (Currently under MIT License) at <https://github.com/cta-observatory/ctapipe>

* We use the "Github-Workflow": 
  * Few people (e.g. @kosack, @mackaiver) have write access to the main repository
  * Contributors fork the main repository and work on branches
  * Pull Requests are merged after Code Review and automatic execution of the test suite

* Early developement stage ⇒ backwards-incompatible API changes might and will happen 

* Many open design questions ⇒ Core Developer Meeting in the second week of December in Dortmund

## What's there?

* Reading simtel simulation files
* Simple calibration, cleaning and feature extraction functions
* Camera and Array plotting
* Coordinate frames and transformations 
* Stereo-reconstruction using line intersections
  
 

## What's still missing?

* Easy to use IO of analysis results to standard data formats (e.g. FITS, hdf5)
* Easy to use "analysis builder"
* A "Standard Analysis"
* Good integration with machine learning techniques
* IRF calculation 
* Defining APIs for IO, instrument description access etc.
* Most code only tested on HESSIO simulations
* Documentation, e.g. formal definitions of coordinate frames 
 
 

## What can you do?

* Report issues
  * Hard to get started? Tell us where you are stuck
  * Tell user stories
  * Missing features

* Start contributing
  * ctapipe needs more workpower
  * Implement new reconstruction features

# A simple hillas analysis

## Reading in simtel files

In [ ]:
from ctapipe.io import EventSourceFactory
from ctapipe.utils.datasets import get_dataset_path

input_url = get_dataset_path('gamma_test_large.simtel.gz')

# The EventSourceFactory automatically detects what kind of file we are giving it,
# if already supported by ctapipe
event_source = EventSourceFactory.produce(input_url=input_url, max_events=9)

print(type(event_source))

In [ ]:
for event in event_source:
    print(f'Id: {event.count}', f'E = {event.mc.energy:1.3f}', f'Telescopes: {len(event.r0.tel)}', sep=', ')

Each event is a `DataContainer` holding several `Field`s of data, which can be containers or just numbers.
Let's look a one event:

In [ ]:
event

In [ ]:
event.inst.subarray.camera_types

In [ ]:
len(event.r0.tel), len(event.r1.tel)

## Data calibration

As we saw, the data container only contains raw data (only the `r0` containers are filled)

So we use the `CameraCalibrator` factory to calibrate the event.

In [ ]:
from ctapipe.calib import CameraCalibrator

calibrator = CameraCalibrator(
    eventsource=event_source,
)

In [ ]:
calibrator.r1

In [ ]:
calibrator.calibrate(event)

## Event displays

Let's use ctapipe's plotting facilities to plot the telescope images

In [ ]:
event.dl1.tel.keys()

In [ ]:
tel_id = 15


In [ ]:
camera = event.inst.subarray.tel[tel_id].camera
dl1 = event.dl1.tel[tel_id]

camera, dl1

In [ ]:
from ctapipe.visualization import CameraDisplay

display = CameraDisplay(camera)

# right now, there might be one image per gain channel.
# This will change as soon as 
display.image = dl1.image[0]
display.add_colorbar()

## Image Cleaning

In [ ]:
from ctapipe.image.cleaning import tailcuts_clean

In [ ]:
# unoptimized cleaning levels, copied from 
# https://github.com/tudo-astroparticlephysics/cta_preprocessing
cleaning_level = {
    'ASTRICam': (5, 7, 2),  # (5, 10)?
    'LSTCam': (3.5, 7.5, 2),  # ?? (3, 6) for Abelardo...
    'FlashCam': (4, 8, 2),  # there is some scaling missing?
}

In [ ]:
boundary, picture, min_neighbors = cleaning_level[camera.cam_id]

clean = tailcuts_clean(
    camera, 
    dl1.image[0],
    boundary_thresh=boundary,
    picture_thresh=picture,
    min_number_picture_neighbors=min_neighbors
)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

d1 = CameraDisplay(camera, ax=ax1)
d2 = CameraDisplay(camera, ax=ax2)

ax1.set_title('Image')
d1.image = dl1.image[0]
d1.add_colorbar(ax=ax1)

ax2.set_title('PeakPos')
d2.image = dl1.peakpos[0] - np.average(dl1.peakpos[0], weights=dl1.image[0])
d2.cmap = 'RdBu_r'
d2.add_colorbar(ax=ax2)

d1.highlight_pixels(clean, color='red', linewidth=1)

## Image Parameters

In [ ]:
from ctapipe.image import hillas_parameters, leakage, concentration
from ctapipe.image.timing_parameters import timing_parameters
from ctapipe.image.cleaning import number_of_islands

In [ ]:
hillas = hillas_parameters(camera[clean], dl1.image[0][clean])

print(hillas)

In [ ]:
display = CameraDisplay(camera)

# set "unclean" pixels to 0
cleaned = dl1.image[0].copy()
cleaned[~clean] = 0.0

display.image = cleaned
display.add_colorbar()

display.overlay_moments(hillas, color='xkcd:red')

In [ ]:
timing = timing_parameters(
    camera[clean],
    dl1.image[0][clean],
    dl1.peakpos[0][clean],
    hillas,
)

print(timing)

In [ ]:
l = leakage(camera, dl1.image[0], clean)
print(l)

In [ ]:
n_islands, island_id = number_of_islands(camera, clean)

print(n_islands)

In [ ]:
conc = concentration(camera, dl1.image[0], hillas)
print(conc)

## Putting it all together / Stereo reconstruction

In [ ]:
import astropy.units as u

from ctapipe.io import EventSourceFactory
from ctapipe.utils.datasets import get_dataset_path

from ctapipe.calib import CameraCalibrator

from ctapipe.image.cleaning import tailcuts_clean, number_of_islands

from ctapipe.reco import HillasReconstructor


# unoptimized cleaning levels, copied from 
# https://github.com/tudo-astroparticlephysics/cta_preprocessing
cleaning_level = {
    'ASTRICam': (5, 7, 2),  # (5, 10)?
    'LSTCam': (3.5, 7.5, 2),  # ?? (3, 6) for Abelardo...
    'FlashCam': (4, 8, 2),  # there is some scaling missing?
}


input_url = get_dataset_path('gamma_test_large.simtel.gz')
event_source = EventSourceFactory.produce(input_url=input_url, max_events=4)


calibrator = CameraCalibrator(
    eventsource=event_source,
)

reco = HillasReconstructor()

for event in event_source:
    print(f'Id: {event.count}', f'E = {event.mc.energy:1.3f}', f'Telescopes: {len(event.r0.tel)}', sep=', ')
    
    calibrator.calibrate(event)
    
    # mapping of telescope_id to parameters for stereo reconstruction
    hillas_containers = {}
    pointing_azimuth = {}
    pointing_altitude = {}
    time_gradients = {}
    
    for telescope_id, dl1 in event.dl1.tel.items():
        camera = event.inst.subarray.tels[telescope_id].camera
        image = dl1.image[0]
        peakpos = dl1.peakpos[0]
        
        boundary, picture, min_neighbors = cleaning_level[camera.cam_id]

        clean = tailcuts_clean(
            camera, 
            image,
            boundary_thresh=boundary,
            picture_thresh=picture,
            min_number_picture_neighbors=min_neighbors
        )
        
        hillas_c = hillas_parameters(camera[clean], image[clean])
        leakage_c = leakage(camera, image, clean)
        n_islands, island_ids = number_of_islands(camera, clean)
        
        timing_c = timing_parameters(camera[clean], image[clean], peakpos[clean], hillas_c)
        
        hillas_containers[telescope_id] = hillas_c
        
        # ssts have no timing in prod4, so we'll use the skewness
        time_gradients[telescope_id] = timing_c.slope.value if camera.cam_id != 'ASTRICam' else hillas_c.skewness
        
        pointing_azimuth[telescope_id] = event.mc.tel[telescope_id].azimuth_raw * u.rad
        pointing_altitude[telescope_id] = event.mc.tel[telescope_id].altitude_raw * u.rad
        
    stereo = reco.predict(
        hillas_containers, event.inst, pointing_altitude, pointing_azimuth
    )
    
    print(f'  Alt: {stereo.alt.deg:.2f}°')
    print(f'  Az: {stereo.az.deg:.2f}°')
    print(f'  Hmax: {stereo.h_max:.0f}')
    print(f'  CoreX: {stereo.core_x:.1f}')
    print(f'  CoreY: {stereo.core_y:.1f}')



## ArrayDisplay


In [ ]:
from ctapipe.visualization import ArrayDisplay


angle_offset = event.mcheader.run_array_direction[0]


disp = ArrayDisplay(event.inst.subarray)

disp.set_vector_hillas(hillas_containers, time_gradient=time_gradients, angle_offset=angle_offset, length=500)
plt.scatter(event.mc.core_x, event.mc.core_y, s=200, c='k', marker='x', label='True Impact',)
plt.scatter(stereo.core_x, stereo.core_y, s=200, c='r', marker='x', label='Estimated Impact',)

plt.legend()
plt.xlim(-400, 400)
plt.ylim(-400, 400)

# LST Mono with output


* Let's use the `HDF5TableWriter` to save the dl2 data to an hdf5 file
* This is not ideal yet and one of the major points to be discussed in two weeks

In [ ]:
from ctapipe.io import HDF5TableWriter


input_url = get_dataset_path('gamma_test_large.simtel.gz')

event_source = EventSourceFactory.produce(
    input_url=input_url,
    allowed_tels=[1, 2, 3, 4], # only use the first LST
)


calibrator = CameraCalibrator(
    eventsource=event_source,
)


with HDF5TableWriter(filename='hillas.h5', group_name='dl2', mode='w') as writer:

    for event in event_source:
        print(f'Id: {event.count}', f'E = {event.mc.energy:1.3f}', f'Telescopes: {len(event.r0.tel)}', sep=', ')
    
        calibrator.calibrate(event)
    
        for telescope_id, dl1 in event.dl1.tel.items():      

            camera = event.inst.subarray.tels[telescope_id].camera
            image = dl1.image[0]
            peakpos = dl1.peakpos[0]

            boundary, picture, min_neighbors = cleaning_level[camera.cam_id]

            clean = tailcuts_clean(
                camera, 
                image,
                boundary_thresh=boundary,
                picture_thresh=picture,
                min_number_picture_neighbors=min_neighbors
            )
            
            if clean.sum() < 5:
                continue

            hillas_c = hillas_parameters(camera[clean], image[clean])
            leakage_c = leakage(camera, image, clean)
            timing_c = timing_parameters(camera[clean], image[clean], peakpos[clean], hillas_c)

            writer.write('events', [event, event.mc, hillas_c, leakage_c, timing_c])
    


In [ ]:
df = pd.read_hdf('hillas.h5')

print(len(df))
df.head()

In [ ]:
plt.scatter(np.log10(df.energy), np.log10(df.intensity))
plt.xlabel('log10(E / TeV)')
plt.ylabel('log10(intensity)')
None